In [65]:
import org.apache.spark.sql.{Row, DataFrame}

val myDF: DataFrame = spark.read.format("csv").option("header", "true").load("../dat/crypto_clean.csv")

In [66]:
var rdd = myDF.select("tags").rdd.map{case Row(t: String) => t}

In [73]:
val tags = rdd.collect.flatMap(_.split(" ").map(x => {
        val numWords = x.split("-").size
        (x, numWords)
    }))

In [68]:
tags.maxBy(_._2)

(meet-in-the-middle-attack,5)

### TODO: load all data and do a histogram over tag lengths

In [69]:
val anyNull = myDF.columns.map(myDF(_).isNull).reduce(_||_)
myDF.filter(anyNull).show

+---+-----+-------+----+
| id|title|content|tags|
+---+-----+-------+----+
+---+-----+-------+----+



In [72]:
import org.apache.spark.sql.functions._
myDF.filter(length(myDF("tags")) > 30).show

+---+--------------------+--------------------+--------------------+
| id|               title|             content|                tags|
+---+--------------------+--------------------+--------------------+
|  7|Why use a 1-2 Obl...|<p>When initiatin...|oblivious-transfe...|
|  9|What is the gener...|<p>Since most cry...|hash cryptanalysi...|
| 17|How to provide se...|<p><a href="http:...|dsa bitcoin digit...|
| 20|What are the prac...|<p>AES has severa...|encryption aes bl...|
| 27|How do I apply di...|<p>I have read a ...|cryptanalysis blo...|
| 31|Current mathemati...|<p>What are the m...|coding-theory ell...|
| 36|What are the main...|<p>What are the m...|cryptanalysis cla...|
| 48|Is it feasible to...|<p>A few years ag...|encryption hash b...|
| 55|Known methods for...|<p>There are seve...|implementation ae...|
| 59|Taking advantage ...|<p>Suppose Alice ...|cryptanalysis one...|
| 66|What are the deta...|<p>I think I once...|encryption des mo...|
| 71|How can I generat...|<p>What 

### Cleaning Steps
Based of some initial experimentation it looks like the following steps (in order) are need to get the data imported properly:

- Remove inter-row newlines. This can be done with `../fix_newlines.sh`
- Replace all `"""` instances with `" ""`. This handles the case where a post starts with quotations.
- Replace all `""` with `\"`. The double quotes confuse the heck out of Spark's csv loader

In [80]:
val ss = org.apache.spark.sql.SparkSession.builder.getOrCreate()
import ss.implicits._

In [83]:
myDF.as[(String, String, String, String)].sho

+---+--------------------+--------------------+--------------------+
| id|               title|             content|                tags|
+---+--------------------+--------------------+--------------------+
|  3|What are the bene...|<p>Why do we use ...|block-cipher des ...|
|  7|Why use a 1-2 Obl...|<p>When initiatin...|oblivious-transfe...|
|  8|Why do we append ...|<p>As we know, <a...|          sha-1 hash|
|  9|What is the gener...|<p>Since most cry...|hash cryptanalysi...|
| 14|How can I use asy...|<p>RSA is not des...|encryption rsa pu...|
| 16|How were the DES ...|<p>It seems like ...|des encryption s-...|
| 17|How to provide se...|<p><a href="http:...|dsa bitcoin digit...|
| 18|How can a random ...|<p>I understand t...|hash implementati...|
| 20|What are the prac...|<p>AES has severa...|encryption aes bl...|
| 24|What makes a hash...|<p>Using a crypto...|      hash passwords|
| 27|How do I apply di...|<p>I have read a ...|cryptanalysis blo...|
| 31|Current mathemati...|<p>What 